In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, results as rs

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201210.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-0912.csv',names = ["timestamp","Pick Station","Availability","Blue Tote Loss","Grey Tote Loss"])
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'Faults20_11-10_12.csv')

In [4]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=2250218506272) was called 1 times


2021-01-05 18:38:27,436 - NumExpr defaulting to 8 threads.


In [5]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2250218507136) was called 1 times


In [14]:
fa,unmapped, end_time = feat.preprocess_faults(scs_raw,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2250218506128) was called 2 times
Running: Function "load_tote_lookup" (id=2250218507856) was called 2 times
Running: Function "load_module_lookup" (id=2250215668608) was called 2 times
Running: Function "load_ID_lookup" (id=2250218504544) was called 2 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [15]:
fa_floor = feat.floor_shift_time_fa(fa, shift=15)

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:294: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fa_floor['Duration'][i] = fa_floor['Time Left'][i]
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fa_floor['Duration'][i+1] = fa_floor['Duration'][i+1] - fa_floor['Duration'][i]
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:291: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

In [16]:
df,fa_PTT = feat.create_PTT_df(fa_floor,at,av)

df = feat.log_totes(df) 

Running: Function "load_PTT_lookup" (id=2250218506560) was called 37 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2250218505984) was called 37 times
Running: Function "load_PTT_lookup" (id=2250218506560) was called 38 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2250218505984) was called 38 times
Running: Function "load_PTT_lookup" (id=2250218506560) was called 39 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2250218505984) was called 39 times
Running: Function "load_PTT_lookup" (id=2250218506560) was called 40 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2250218505984) was called 40 times
Running: Function "load_PTT_lookup" (id=2250218506560) was called 41 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2250218505984) was called 41 times
Running: Function "load_PTT_lookup" (id=2250218506560) was called 42 times
Previous Hours Weigh

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:704: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['log_totes'] = np.log(df['TOTES'])


In [17]:
df_week = df[df['timestamp']>end_time - pd.to_timedelta(14, unit='D')]

In [18]:
X,y = md.gen_feat_var(df_week,target = 'Availability', features = ['Faults','Totes'])


In [19]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': None})

R2_cv,R2_OOS,Coeff = md.run_OLS(X_train = X_train,y_train = y_train,X_test = X_test,y_test=y_test, n = 30)

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)



Cross Validation Scores: 
 
        R2 Scores
1       0.455477
2       0.519002
3       0.479313
4       0.430159
5       0.523638
Mean    0.481518
STD     0.036055
232
                                 OLS Regression Results                                
Dep. Variable:           Availability   R-squared (uncentered):                   0.960
Model:                            OLS   Adj. R-squared (uncentered):              0.959
Method:                 Least Squares   F-statistic:                              862.4
Date:                Tue, 05 Jan 2021   Prob (F-statistic):                        0.00
Time:                        18:44:42   Log-Likelihood:                          4908.5
No. Observations:                8152   AIC:                                     -9373.
Df Residuals:                    7930   BIC:                                     -7818.
Df Model:                         222                                                  
Covariance Type:            nonrobust 

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [20]:
R2_OOS

0.3697563257316707

In [ ]:
Output = rs.create_output(fa_PTT,Coeff,end_time)

In [ ]:
Output['Total'] = sum(Output[:][Output.columns[9:]])

In [ ]:
Output[Output.columns[9]]

In [ ]:
sns.lmplot(x='log_totes', y='Availability', data=df, line_kws={"color": "red"})
plt.xlim(0, 5)

In [ ]:
df.shape

In [ ]:
sns.lmplot(x='TOTES', y='Availability', data=df, line_kws={"color": "red"})

In [ ]:
ax = sns.regplot(x="TOTES", y="Availability", data=df,
x_estimator=np.mean, logx=True, line_kws={"color": "red"})
plt.ylim(0,1)

In [ ]:
sns.lmplot(x='log_totes', y='Availability', data=df, line_kws={"color": "red"})
plt.xlim(0, 5)